In [16]:
import requests
import os
import pandas as pd

def extract_data(data, prefix=''):
    result = {}
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{prefix}{key}" if prefix else key
            if isinstance(value, (dict, list)):
                result.update(extract_data(value, new_key + '.'))
            else:
                result[new_key] = value
    elif isinstance(data, list):
        for index, value in enumerate(data):
            new_key = f"{prefix}{index}" if prefix else str(index)
            if isinstance(value, (dict, list)):
                result.update(extract_data(value, new_key + '.'))
            else:
                result[new_key] = value
    return result
    

def akv_query_FS_graphql(query, variable):
    """
    Send a GraphQL query to Felles Studentsystem and return the response.

    :param query: GraphQL query
    :type query: str
    :param variable: GraphQL variable
    :type variable: dict
    :return: JSON response
    :rtype: dict
    :raises Exception: if the request fails
    """
    hode = {
        'Accept': 'application/json;version=1',
        'Authorization': f'Basic {os.environ["tokenFS"]}',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode)
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return {}


query = """
query MyQuery($antal: Int, $start: String) {
    programStudieretter(
        filter: {eierOrganisasjonskode: "0203", aktivStatus: AKTIV}, 
        after: $start,
        first: $antal) {
        pageInfo {
            endCursor
            hasNextPage
        }
        nodes {
            personProfil {
                personlopenummer
            }
            studieprogram {
                kode
            }
            campus {
                navnAlleSprak {
                    nb
                }
            }
            kull {
                termin {
                    arstall
                    betegnelse {
                        navnAlleSprak {
                            nb
                        }
                    }
                }
            }
        }
    }
}
"""


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [14]:
antal_per_side = 5
start = ''
variable = {'antal': antal_per_side, 'start': start}
svar = akv_query_FS_graphql(query, variable)

In [27]:
data = extract_data(svar['data']['programStudieretter']['nodes'])
data

{'0.personProfil.personlopenummer': '65',
 '0.studieprogram.kode': 'HSPED',
 '0.campus.navnAlleSprak.nb': 'Stord',
 '0.kull.termin.arstall': 2023,
 '0.kull.termin.betegnelse.navnAlleSprak.nb': 'Høst',
 '1.personProfil.personlopenummer': '252',
 '1.studieprogram.kode': 'MPUV',
 '1.campus.navnAlleSprak.nb': 'Bergen',
 '1.kull.termin.arstall': 2023,
 '1.kull.termin.betegnelse.navnAlleSprak.nb': 'Vår',
 '2.personProfil.personlopenummer': '347',
 '2.studieprogram.kode': 'B-KVL',
 '2.campus.navnAlleSprak.nb': 'Bergen',
 '2.kull.termin.arstall': 2024,
 '2.kull.termin.betegnelse.navnAlleSprak.nb': 'Høst',
 '3.personProfil.personlopenummer': '360',
 '3.studieprogram.kode': 'HSPED',
 '3.campus.navnAlleSprak.nb': 'Stord',
 '3.kull.termin.arstall': 2024,
 '3.kull.termin.betegnelse.navnAlleSprak.nb': 'Høst',
 '4.personProfil.personlopenummer': '496',
 '4.studieprogram.kode': 'MPUV',
 '4.campus.navnAlleSprak.nb': 'Bergen',
 '4.kull.termin.arstall': 2023,
 '4.kull.termin.betegnelse.navnAlleSprak.nb':

In [28]:
dr = pd.DataFrame.from_dict(data, orient="index")
dr

,0
0.personProfil.personlopenummer,65
0.studieprogram.kode,HSPED
0.campus.navnAlleSprak.nb,Stord
0.kull.termin.arstall,2023
0.kull.termin.betegnelse.navnAlleSprak.nb,Høst
1.personProfil.personlopenummer,252
1.studieprogram.kode,MPUV
1.campus.navnAlleSprak.nb,Bergen
1.kull.termin.arstall,2023
1.kull.termin.betegnelse.navnAlleSprak.nb,Vår


In [26]:
svar['data']['programStudieretter']['nodes']

[{'personProfil': {'personlopenummer': '65'},
  'studieprogram': {'kode': 'HSPED'},
  'campus': {'navnAlleSprak': {'nb': 'Stord'}},
  'kull': {'termin': {'arstall': 2023,
    'betegnelse': {'navnAlleSprak': {'nb': 'Høst'}}}}},
 {'personProfil': {'personlopenummer': '252'},
  'studieprogram': {'kode': 'MPUV'},
  'campus': {'navnAlleSprak': {'nb': 'Bergen'}},
  'kull': {'termin': {'arstall': 2023,
    'betegnelse': {'navnAlleSprak': {'nb': 'Vår'}}}}},
 {'personProfil': {'personlopenummer': '347'},
  'studieprogram': {'kode': 'B-KVL'},
  'campus': {'navnAlleSprak': {'nb': 'Bergen'}},
  'kull': {'termin': {'arstall': 2024,
    'betegnelse': {'navnAlleSprak': {'nb': 'Høst'}}}}},
 {'personProfil': {'personlopenummer': '360'},
  'studieprogram': {'kode': 'HSPED'},
  'campus': {'navnAlleSprak': {'nb': 'Stord'}},
  'kull': {'termin': {'arstall': 2024,
    'betegnelse': {'navnAlleSprak': {'nb': 'Høst'}}}}},
 {'personProfil': {'personlopenummer': '496'},
  'studieprogram': {'kode': 'MPUV'},
  'ca